In [31]:
using GeometryTypes
using LCMGL

In [11]:
import SoftRobots

In [57]:
reload("SoftRobots")

In [58]:
function cube()
    k = 10.
    b = 4.0
    l = 1.0
    m = 0.1
    nodes = SoftRobots.PointMass[]
    positions = SoftRobots.Point{3, Float64}[]
    for z = 0:1
        for x = 0.1:1.1
            for y = -1:0
                push!(nodes, SoftRobots.PointMass(m))
                push!(positions, SoftRobots.Point{3, Float64}(x, y, z))
            end
        end
    end
    num_nodes = length(nodes)
    edges = SoftRobots.DampedSpring[]
    for i = 1:num_nodes
        for j = (i+1):num_nodes
            rest_length = norm(positions[i] - positions[j])
            push!(edges, SoftRobots.DampedSpring(i, j, k, b, rest_length))
        end
    end
    faces = SoftRobots.convex_hull(positions)
    robot = SoftRobots.SoftRobot(nodes, edges, faces)
    velocities = [SoftRobots.Point{3, Float64}(0) for n in robot.nodes]
    state = SoftRobots.SoftRobotState(positions, velocities)
    robot, state
end

cube (generic function with 1 method)

In [59]:
robot, state = cube();
SoftRobots.draw(robot, state)

In [60]:
function tower()
    k = 10.
    b = 4.0
    l = 1.0
    m = 0.1
    nodes = SoftRobots.PointMass[]
    positions = SoftRobots.Point{3, Float64}[]
    for z = [0]
        for x = [-0.5; 2.5]
            for y = [-0.5; 5]
                push!(nodes, SoftRobots.PointMass(m))
                push!(positions, SoftRobots.Point{3, Float64}(x - 0.5 * y, y, z))
            end
        end
    end
    for (x,y,z) in Any[(0,0,1.1)]
        push!(nodes, SoftRobots.PointMass(m))
        push!(positions, SoftRobots.Point{3, Float64}(x, y, z))
    end

    num_nodes = length(nodes)
    edges = SoftRobots.DampedSpring[]
    for i = 1:num_nodes
        for j = (i+1):num_nodes
            rest_length = norm(positions[i] - positions[j])
            push!(edges, SoftRobots.DampedSpring(i, j, k, b, rest_length))
        end
    end
    faces = SoftRobots.convex_hull(positions)
    robot = SoftRobots.SoftRobot(nodes, edges, faces)
    velocities = [SoftRobots.Point{3, Float64}(0) for n in robot.nodes]
    state = SoftRobots.SoftRobotState(positions, velocities)
    robot, state
end

tower (generic function with 1 method)

In [61]:
robot, state = tower();
SoftRobots.draw(robot, state)

In [62]:
mesh = HomogenousMesh(state.positions, robot.faces)

LCMGLClient("normals") do lcmgl
    normals = decompose(GeometryTypes.Normal{3, Float64}, mesh)
    color(lcmgl, 0,0,0)
    begin_mode(lcmgl, LCMGL.LINES)
    for i = 1:length(state.positions)
        vertex(lcmgl, state.positions[i]...)
        vertex(lcmgl, (state.positions[i] + Point(normals[i]))...)
    end
    end_mode(lcmgl)
    switch_buffer(lcmgl)
end
        